In [2]:
import os
import glob
import cv2
import numpy as np


def get_video_info(cap: cv2.VideoCapture) -> dict:
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = float(cap.get(cv2.CAP_PROP_FPS)) or 0.0
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 0
    duration = (frame_count / fps) if (fps > 0 and frame_count > 0) else None
    return {
        "width": w,
        "height": h,
        "fps": fps,
        "frame_count": frame_count,
        "duration_sec": duration,
    }


def center_crop_square(frame: np.ndarray, target_size: int | None = None) -> np.ndarray:
    h, w = frame.shape[:2]
    crop = min(w, h)

    x0 = (w - crop) // 2
    y0 = (h - crop) // 2

    out = frame[y0:y0 + crop, x0:x0 + crop]

    if target_size is not None:
        out = cv2.resize(out, (target_size, target_size), interpolation=cv2.INTER_AREA)

    return out


def remove_black_dots_on_white(
    frame_bgr: np.ndarray,
    dark_thresh: int = 200,
    min_area: int = 30,
) -> np.ndarray:
    gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)

    # 어두운 픽셀 마스크 (어두우면 255)
    dark_mask = (gray < dark_thresh).astype(np.uint8) * 255

    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(
        dark_mask, connectivity=8
    )

    out = frame_bgr.copy()
    for lab in range(1, num_labels):
        area = stats[lab, cv2.CC_STAT_AREA]
        if area <= min_area:
            out[labels == lab] = (255, 255, 255)

    return out


def paint_white_border(frame_bgr: np.ndarray, border_px: int = 15) -> np.ndarray:
    """
    프레임의 상/하/좌/우 테두리 border_px 만큼을 흰색으로 덮습니다.
    """
    if border_px <= 0:
        return frame_bgr

    h, w = frame_bgr.shape[:2]
    b = min(border_px, h // 2, w // 2)  # 안전장치

    out = frame_bgr.copy()
    # top
    out[:b, :] = (255, 255, 255)
    # bottom
    out[h - b:h, :] = (255, 255, 255)
    # left
    out[:, :b] = (255, 255, 255)
    # right
    out[:, w - b:w] = (255, 255, 255)

    return out


def process_video(
    input_path: str,
    output_path: str,
    target_size: int | None,
    dark_thresh: int,
    min_area: int,
    border_px: int,
):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise RuntimeError(f"영상을 열 수 없습니다: {input_path}")

    info = get_video_info(cap)

    print(f"\n[INPUT] {os.path.basename(input_path)}")
    print(f"  resolution = {info['width']} x {info['height']}")
    print(f"  fps = {info['fps']:.3f}, frames = {info['frame_count']}")

    out_size = target_size if target_size is not None else min(info["width"], info["height"])
    fps = info["fps"] if info["fps"] > 0 else 30.0

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (out_size, out_size))
    if not out.isOpened():
        cap.release()
        raise RuntimeError(f"VideoWriter 열기 실패: {output_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 1) 정사각형 크롭(+옵션 리사이즈)
        frame_sq = center_crop_square(frame, target_size)

        # 2) 흰 배경 검은 점 제거
        frame_dn = remove_black_dots_on_white(
            frame_sq,
            dark_thresh=dark_thresh,
            min_area=min_area,
        )

        # 3) 테두리 15px(기본) 흰색 덮기
        frame_final = paint_white_border(frame_dn, border_px=border_px)

        out.write(frame_final)

    cap.release()
    out.release()

    print(f"[OUTPUT] {os.path.basename(output_path)}")
    print(f"  resolution = {out_size} x {out_size}")
    print(f"  border_px = {border_px}")


def process_directory(
    input_dir: str,
    output_dir: str,
    target_size: int | None = None,
    dark_thresh: int = 200,
    min_area: int = 30,
    border_px: int = 15,
):
    os.makedirs(output_dir, exist_ok=True)

    video_paths = sorted(glob.glob(os.path.join(input_dir, "*.mp4")))
    if not video_paths:
        raise RuntimeError(f"{input_dir} 에 mp4 파일이 없습니다.")

    print(f"총 {len(video_paths)}개 영상 처리 시작")
    for path in video_paths:
        name = os.path.splitext(os.path.basename(path))[0]
        out_path = os.path.join(output_dir, f"{name}_square_denoise_border.mp4")

        process_video(
            path,
            out_path,
            target_size=target_size,
            dark_thresh=dark_thresh,
            min_area=min_area,
            border_px=border_px,
        )


if __name__ == "__main__":
    # ================== 사용자 설정 ==================
    input_dir = r"./raw"
    output_dir = r"./result"

    target_size = None   # 예: 512 (고정 크기), 아니면 None (원본에서 정사각형)
    dark_thresh = 200
    min_area = 30
    border_px = 15       # 요청하신 값
    # =================================================

    process_directory(
        input_dir=input_dir,
        output_dir=output_dir,
        target_size=target_size,
        dark_thresh=dark_thresh,
        min_area=min_area,
        border_px=border_px,
    )


총 10개 영상 처리 시작

[INPUT] rest_game_sleep_penguin.mp4
  resolution = 1280 x 720
  fps = 24.000, frames = 192
[OUTPUT] rest_game_sleep_penguin_square_denoise_border.mp4
  resolution = 720 x 720
  border_px = 15

[INPUT] rest_game_sleep_penguin_2.mp4
  resolution = 1280 x 720
  fps = 24.000, frames = 192
[OUTPUT] rest_game_sleep_penguin_2_square_denoise_border.mp4
  resolution = 720 x 720
  border_px = 15

[INPUT] rest_phone_penguin.mp4
  resolution = 1280 x 720
  fps = 24.000, frames = 192
[OUTPUT] rest_phone_penguin_square_denoise_border.mp4
  resolution = 720 x 720
  border_px = 15

[INPUT] rest_sunglasses_penguin.mp4
  resolution = 1280 x 720
  fps = 24.000, frames = 192
[OUTPUT] rest_sunglasses_penguin_square_denoise_border.mp4
  resolution = 720 x 720
  border_px = 15

[INPUT] study_penguin.mp4
  resolution = 1280 x 720
  fps = 24.000, frames = 192
[OUTPUT] study_penguin_square_denoise_border.mp4
  resolution = 720 x 720
  border_px = 15

[INPUT] study_penguin_2.mp4
  resolution = 12